# Mistplay Fraud Demo – Data Generation

This notebook generates small, synthetic datasets for a gaming fraud detection demo. The data is intentionally compact to keep runtime short while still enabling realistic joins and aggregations.


## Dataset Description

This notebook generates synthetic datasets for a gaming fraud detection demo. The datasets include:

### Accounts Table

Generates NUM_ACCOUNTS fake user accounts.
Randomizes fields like country, platform, marketing channel, and account creation date.
Assigns a fraud label (is_fraud_label) with higher probability for certain countries (IN, BR).
## Devices Table

Generates NUM_DEVICES fake devices.
Randomizes device type, OS version, and whether it’s an emulator.
Assigns a device risk score (higher for emulators).
## Account-Device Mapping

Randomly assigns a primary device to each account.
## Events Table

For each account, generates a random number of events (more for fraud accounts).
Each event includes timestamp, type, session length, VPN usage, and IP country (with more anomalies for fraud accounts).
## Rewards Table

For each account, generates a random number of rewards (more for fraud accounts).
Each reward includes timestamp, type, amount, and redemption channel.

In [0]:
from pyspark.sql import functions as F

# Demo configuration
DB_NAME = "ramin_serverless_aws_catalog.mistplay_fraud_demo"
NUM_ACCOUNTS = 5000
NUM_DEVICES = 3000
MAX_EVENT_DAYS = 7

spark.sql(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
spark.sql(f"USE {DB_NAME}")
spark.conf.set("spark.sql.shuffle.partitions", "8")

countries = ["US", "CA", "GB", "DE", "FR", "BR", "IN", "JP", "KR", "AU"]
platforms = ["android", "ios"]
marketing_channels = ["organic", "ad_network", "influencer", "affiliate"]
device_types = ["phone", "tablet"]
os_versions = ["12", "13", "14", "15", "16"]
event_types = ["session_start", "level_complete", "store_open", "reward_claim"]
reward_types = ["coins", "gems", "bonus_boost"]

countries_arr = F.array([F.lit(x) for x in countries])
platforms_arr = F.array([F.lit(x) for x in platforms])
marketing_arr = F.array([F.lit(x) for x in marketing_channels])
device_types_arr = F.array([F.lit(x) for x in device_types])
os_versions_arr = F.array([F.lit(x) for x in os_versions])
event_types_arr = F.array([F.lit(x) for x in event_types])
reward_types_arr = F.array([F.lit(x) for x in reward_types])

# Accounts table
accounts = (
    spark.range(NUM_ACCOUNTS)
    .withColumn("account_id", F.col("id").cast("string"))
    .withColumn("created_date", F.expr("date_sub(current_date(), cast(rand() * 180 as int))"))
    .withColumn(
        "country",
        F.element_at(countries_arr, (F.floor(F.rand() * len(countries)) + 1).cast("int")),
    )
    .withColumn(
        "platform",
        F.element_at(platforms_arr, (F.floor(F.rand() * len(platforms)) + 1).cast("int")),
    )
    .withColumn(
        "marketing_channel",
        F.element_at(marketing_arr, (F.floor(F.rand() * len(marketing_channels)) + 1).cast("int")),
    )
)

fraud_prob = F.when(F.col("country").isin("IN", "BR"), F.lit(0.05)).otherwise(F.lit(0.02))
accounts = accounts.withColumn("is_fraud_label", (F.rand() < fraud_prob).cast("int"))

# Devices table
devices = (
    spark.range(NUM_DEVICES)
    .withColumn("device_id", F.col("id").cast("string"))
    .withColumn(
        "device_type",
        F.element_at(device_types_arr, (F.floor(F.rand() * len(device_types)) + 1).cast("int")),
    )
    .withColumn(
        "os_version",
        F.element_at(os_versions_arr, (F.floor(F.rand() * len(os_versions)) + 1).cast("int")),
    )
    .withColumn("is_emulator", (F.rand() < 0.03).cast("int"))
    .withColumn(
        "device_risk_score",
        F.when(F.col("is_emulator") == 1, F.rand() * 0.2 + 0.8).otherwise(F.rand() * 0.6),
    )
)

# Account-device mapping (primary device)
account_device = (
    accounts.select("account_id", "is_fraud_label")
    .withColumn("device_id", (F.floor(F.rand() * NUM_DEVICES)).cast("int").cast("string"))
)

# Events table
events_seed = (
    accounts.select("account_id", "country", "is_fraud_label")
    .withColumn(
        "event_count",
        F.when(
            F.col("is_fraud_label") == 1,
            F.expr("cast(30 + rand() * 70 as int)"),
        ).otherwise(F.expr("cast(5 + rand() * 25 as int)")),
    )
)

events = (
    events_seed
    .withColumn("event_idx", F.explode(F.sequence(F.lit(1), F.col("event_count"))))
    .join(account_device.select("account_id", "device_id"), on="account_id", how="left")
    .withColumn(
        "event_ts",
        F.from_unixtime(
            F.unix_timestamp() - (F.rand() * MAX_EVENT_DAYS * 24 * 3600)
        ).cast("timestamp"),
    )
    .withColumn(
        "event_type",
        F.element_at(event_types_arr, (F.floor(F.rand() * len(event_types)) + 1).cast("int")),
    )
    .withColumn(
        "session_minutes",
        F.when(F.col("is_fraud_label") == 1, F.rand() * 10 + 1).otherwise(F.rand() * 40 + 5),
    )
    .withColumn(
        "is_vpn",
        F.when(F.col("is_fraud_label") == 1, (F.rand() < 0.6).cast("int")).otherwise(
            (F.rand() < 0.1).cast("int")
        ),
    )
    .withColumn(
        "ip_country",
        F.when(
            (F.col("is_fraud_label") == 1) & (F.rand() < 0.7),
            F.element_at(countries_arr, (F.floor(F.rand() * len(countries)) + 1).cast("int")),
        ).otherwise(F.col("country")),
    )
    .drop("event_count", "event_idx")
)

# Rewards table
rewards_seed = accounts.select("account_id", "is_fraud_label").withColumn(
    "reward_count",
    F.when(
        F.col("is_fraud_label") == 1,
        F.expr("cast(5 + rand() * 20 as int)"),
    ).otherwise(F.expr("cast(1 + rand() * 6 as int)")),
)

rewards = (
    rewards_seed
    .withColumn("reward_idx", F.explode(F.sequence(F.lit(1), F.col("reward_count"))))
    .withColumn(
        "reward_ts",
        F.from_unixtime(
            F.unix_timestamp() - (F.rand() * MAX_EVENT_DAYS * 24 * 3600)
        ).cast("timestamp"),
    )
    .withColumn(
        "reward_type",
        F.element_at(reward_types_arr, (F.floor(F.rand() * len(reward_types)) + 1).cast("int")),
    )
    .withColumn(
        "reward_amount",
        F.when(F.col("is_fraud_label") == 1, F.rand() * 30 + 10).otherwise(F.rand() * 10 + 1),
    )
    .withColumn(
        "redemption_channel",
        F.when(F.rand() < 0.7, F.lit("in_app")).otherwise(F.lit("web")),
    )
    .drop("reward_count", "reward_idx")
)

# Write Delta tables
accounts.write.mode("overwrite").saveAsTable(f"{DB_NAME}.accounts")
devices.write.mode("overwrite").saveAsTable(f"{DB_NAME}.devices")
account_device.write.mode("overwrite").saveAsTable(f"{DB_NAME}.account_device")
events.write.mode("overwrite").saveAsTable(f"{DB_NAME}.events")
rewards.write.mode("overwrite").saveAsTable(f"{DB_NAME}.rewards")

print("Tables created:")
for name in ["accounts", "devices", "account_device", "events", "rewards"]:
    count = spark.table(f"{DB_NAME}.{name}").count()
    print(f"- {name}: {count} rows")